In [2]:
import pandas as pd
import re

#df : 스크래핑한 호주 문서 30개 테스트
df = pd.read_csv("호주_세관_전체.csv")
df = df[::-1]
df1 = list(df['title'])[:30]
df2 = list(df['text'])[:30]
df = []
for i in range(len(df1)):
    df.append(df1[i]+""+df2[i])

print(len(df))

#df_keyword : 선정한 호주 키워드 101개
df_keyword = pd.read_csv("호주_번역_100.csv", index_col = False)
print(len(df_keyword))

FileNotFoundError: [Errno 2] No such file or directory: '호주_세관_전체.csv'

In [ ]:
import numpy as np
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# SentenceTransformer : 모델 훈련 위한 라이브러리
# 'distiluse-base-multilingual-cased-v1' : 사용할 모델
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [ ]:
#keyword : 불러온 호주 키워드 중 번역 Column 추출
keyword = list(df_keyword["번역"])
print(len(keyword))

In [ ]:
#키워드 벡터화
keyword_embedding = []

for ele in keyword:
    keyword_embedding.append(model.encode(ele))

print(len(keyword_embedding))

In [ ]:
#호주 문서 10개 embedding
import nltk
import string
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 

doc_embedding = []
doc_words = []
cnt = 1
for doc in df:
    result = []
    sentence = ''
    
    #1. 전처리 전 문자 길이
#     print(f"{cnt}번 문서 전처리 전 문자 길이 : {len(doc)}")
    
    #2. 공백 제거 + strip()
    doc = doc.replace("\n", "").strip()
#     print(f"{cnt}번 문서 공백 제거 후 문자 길이 : {len(doc)}")
    
#     #3. 구두점 제거
#     doc = "".join([i for i in doc if i not in string.punctuation]).strip()
#     print(f"{cnt}번 문서 구두점 제거 후 문자 길이 : {len(doc)}")
    
    #4. 단어 토큰화 > 불용어 제거 > 합치기
    words = word_tokenize(doc)
    stop_words = set(stopwords.words('english'))
    doc=''
    for word in words:
        if word not in stop_words:
            result.append(word)
    for i in range(len(result)):
        doc = doc + " " + result[i]
#     print(f"{cnt}번 문서 불용어 제거 후 문자 길이 : {len(doc)}")
    
    #5. 문서 토큰화
    word_tokens = pos_tag(word_tokenize(doc))

    word_tokens = [t[0] for t in word_tokens if t[1] == "NN" and len(t[0]) > 2]
    for w in word_tokens: 
        if w not in stop_words: 
            result.append(w)
    result = [word.lower() for word in result]
    doc_words.append(result)
    doc_embedding.append(model.encode(result))
    
    #카운트 추가
    print(cnt, )
    cnt += 1 

In [3]:
#키워드와 문서 유사도 비교
doc_keywords = []
for index, doc in enumerate(doc_embedding):
    final_result = []
    for i in range(len(doc)):
        for j in range(len(keyword_embedding)):
            print(doc[i])
            print(keyword_embedding[j])
            break
            distances = cosine_similarity([doc[i]], [keyword_embedding[j]])
            if distances[0][0]>0.9:
                final_result.append(doc_words[index][i])
# distances = cosine_similarity([doc_embedding[0]], candidate_embeddings)
    print(set(final_result))
    doc_keywords.append(set(final_result))

NameError: name 'doc_embedding' is not defined